## 1. [10 pts] Describe the environment in the Nim learning model.

The Nim environment in this case would be the game board. For our Nim game, this would be the three stacks of 10 items.

## 2. [10 pts] Describe the agent(s) in the Nim learning model (Hint, not just the Q-learner).

There would be two agents in the game. In our case, these agents will be two of the automated players (random, guru, q-learner) against eachother.

## 3. [10 pts] Describe the reward and penalty in the Nim learning model.
Currently, there is no penalty in the nim learning model. This is what question 6 is asking. The reward is winning a game, as seen by reward only being passed to the q table when the game ends.

Edit: The penalty is making a move that will lead the opponent to a win

## 4. [10 pts] How many possible states there could be in the Nim game with a maximum of 10 items per pile and 3 piles total?
In this game, there are a maximum of 10+10+10=30 moves. There are at least 3 moves in order to finish the game. So the possible states is 30! - 3! ~ 10^32

## 5. [10 pts] How many possible unique actions could there be for player 1 in the Nim game with 10 items per pile and 3 piles total?
For each stack, play 1 can take 10 items, 9 items, 8 items... so it would be 10! + 10! + 10! = 10,886,400

## 6. [50 pts] Find a way to improve the provided Nim game learning model. Do you think one can beat the Guru player? (Hint: How about penalizing the losses? Hint: It is indeed possible to find a better solution, which improves the way Q-learning updates its Q-table). Do not use a Guru player inside the learning module as this would defeat the purpose. However, playing with a Guru is permitted.

In [250]:
import numpy as np
import pandas as pd
from random import randint, choice

# The number of piles is 3


# max number of items per pile
ITEMS_MX = 10

# Initialize starting position
def init_game():
    return [randint(1,ITEMS_MX), randint(1,ITEMS_MX), randint(1,ITEMS_MX)]

# Based on X-oring the item counts in piles - mathematical solution
def nim_guru(st):
    xored = st[0] ^ st[1] ^ st[2]
    if xored == 0:
        return nim_random(st)
    #
    for pile in range(3):
        s = st[pile] ^ xored
        if s <= st[pile]:
            return st[pile]-s, pile

# Random Nim player
def nim_random(_st):
    pile = choice([i for i in range(3) if _st[i]>0])  # find the non-empty piles
    return randint(1, _st[pile]), pile  # random move

def nim_qlearner(_st):
    # pick the best rewarding move, equation 1
    a = np.argmax(qtable[_st[0], _st[1], _st[2]])  # exploitation
    # index is based on move, pile
    move, pile = a%ITEMS_MX+1, a//ITEMS_MX
    # check if qtable has generated a random but game illegal move - we have not explored there yet
    if move <= 0 or _st[pile] < move:
        move, pile = nim_random(_st)  # exploration

    return move, pile  # action

In [384]:
qtable, Alpha, Gamma, Reward, Penalty = None, 1.0, 0.8, 100.0,-100.0

# learn from _n games, randomly played to explore the possible states
def nim_qlearn(_n):
    global qtable
    # based on max items per pile
    qtable = np.zeros((ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX+1, ITEMS_MX*3), dtype=float)
    # play _n games
    for i in range(_n):
        # first state is starting position
        st1 = init_game()
        while True:  # while game not finished
            # make a random move - exploration
            move, pile = nim_random(st1)
            st2 = list(st1)
            # make the move
            st2[pile] -= move  # --> last move I made
            if st2 == [0, 0, 0]:  # game ends
                qtable_update(Reward, st1, move, pile, 0)  # I won
                break  # new game
            else:
                st3 = list(st2)
                move2, pile2 = nim_guru(st2)
                st3[pile2] -=move2
                if st3 == [0,0,0]:
                    qtable_update(Penalty, st1, move, pile, 0)
            
            
            qtable_update(0, st1, move, pile, np.max(qtable[st2[0], st2[1], st2[2]]))
            st1 = st2

# Equation 3 - update the qtable
def qtable_update(r, _st1, move, pile, q_future_best):
    a = pile*ITEMS_MX+move-1
    qtable[_st1[0], _st1[1], _st1[2], a] = Alpha * (r + Gamma * q_future_best)

In [385]:
Engines = {'Random':nim_random, 'Guru':nim_guru, 'Qlearner':nim_qlearner}

def game(a, b):
    state, side = init_game(), 'A'
    while True:
        engine = Engines[a] if side == 'A' else Engines[b]
        move, pile = engine(state)
        # print(state, move, pile)  # debug purposes
        state[pile] -= move
        if state == [0, 0, 0]:  # game ends
            return side  # winning side

        side = 'B' if side == 'A' else 'A'  # switch sides

def play_games(_n, a, b):
    from collections import defaultdict
    wins = defaultdict(int)
    for i in range(_n):
        wins[game(a, b)] += 1
    # info
    print(f"{_n} games, {a:>8s}{wins['A']:5d}  {b:>8s}{wins['B']:5d}")

    return wins['A'], wins['B']

In [386]:
nim_qlearn(10000)

Really don't know what I am doing wrong, I am penalizing the move that leads to an opponent win, but I am not able to come close to beating Guru. I still am able to beat random and qlearner.

In [388]:
play_games(1000, 'Qlearner', 'Random')
play_games(1000, 'Qlearner', 'Qlearner')
play_games(1000, 'Qlearner', 'Guru')

1000 games, Qlearner  696    Random  304
1000 games, Qlearner  901  Qlearner   99
1000 games, Qlearner   19      Guru  981


(19, 981)

In [370]:
def qtable_log(_fn):
    with open(_fn, 'w') as fout:
        s = 'state'
        for a in range(ITEMS_MX*3):
            move, pile = a%ITEMS_MX+1, a//ITEMS_MX
            s += ',%02d_%01d' % (move,pile)

        print(s, file=fout)
        for i, j, k in [(i,j,k) for i in range(ITEMS_MX+1) for j in range(ITEMS_MX+1) for k in range(ITEMS_MX+1)]:
            s = '%02d_%02d_%02d' % (i,j,k)
            for a in range(ITEMS_MX*3):
                r = qtable[i, j, k, a]
                s += ',%.1f' % r

            print(s, file=fout)

qtable_log('qtable_debug.csv')
data = pd.read_csv('qtable_debug.csv')
data


state  01_0  02_0  03_0  04_0  05_0  06_0  07_0  08_0  09_0  ...  \
0     00_00_00   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
1     00_00_01   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
2     00_00_02   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
3     00_00_03   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
4     00_00_04   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
...        ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
1326  10_10_06   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
1327  10_10_07   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
1328  10_10_08   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
1329  10_10_09   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   
1330  10_10_10   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0  ...   

       01_2   02_2   03_2   04_2  05_2  06_2  07_2  08_2  09_2  10_2  
0       0.0    0.0    0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1     100.0    0.0    0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  
2       0.0  100.0    0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  
3       0.0    0.0 -100.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  
4       0.0    0.0    0.0 -100.0   0.0   0.0   0.0   0.0   0.0   0.0  
...     ...    ...    ...    ...   ...   ...   ...   ...   ...   ...  
1326    0.0    0.0    0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1327    0.0    0.0    0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1328    0.0    0.0    0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1329    0.0    0.0    0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  
1330    0.0    0.0    0.0    0.0   0.0   0.0   0.0   0.0   0.0   0.0  

[1331 rows x 31 columns]